In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt = plotter(backend='agg')
%matplotlib inline

In [ ]:
import time
from shapely.geometry import Polygon
from PIL import Image, ImageDraw, ImageFilter
import geopandas as gpd
Image.MAX_IMAGE_PIXELS = 1000000000
from pyproj import Proj
from matplotlib import cm
import scipy.sparse as sp
from scipy import signal
from scipy.ndimage import gaussian_filter

In [ ]:
import location_tools

In [ ]:
def make_rectangle(xlo, xhi, ylo, yhi, poly=True):
    rect = [(xlo, ylo), (xhi, ylo), (xhi, yhi), (xlo, yhi), (xlo, ylo)]
    return Polygon(rect) if poly else rect

## Load Data

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
scenes = pd.read_csv('targets/google_scenes_2002_cloud.csv', parse_dates=['DATE_ACQUIRED'])
print(len(scenes))

## Imagery Coverage

In [ ]:
boxes = scenes[['WEST_LON', 'EAST_LON', 'SOUTH_LAT', 'NORTH_LAT']]
boxes = gpd.GeoSeries(boxes.apply(lambda x: make_rectangle(*x), axis=1))
uboxes = gpd.GeoDataFrame({'geometry': gpd.GeoSeries(boxes.unary_union)})

In [ ]:
lon_min, lon_max, lat_min, lat_max = 55, 150, 0, 70
rect = make_rectangle(lon_min, lon_max, lat_min, lat_max)
area = gpd.GeoDataFrame({'geometry': gpd.GeoSeries(rect)})
crop = gpd.overlay(world.cx[lon_min:lon_max, lat_min:lat_max], area, how='intersection')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4.5))
crop.plot(ax=ax);
# boxes.plot(ax=ax, color='r', alpha=0.2);
uboxes.plot(ax=ax, color='g', alpha=0.2);
plt.axis('equal');

## UTM Coverage

In [ ]:
letters = 'CDEFGHJKLMNPQRSTUVWX'
fletter = {i: c for i, c in enumerate(letters)}
rletter = {c: i for i, c in enumerate(letters)}

def wgs2utm(lon_wgs, lat_wgs):
    idx_lon = int(round(lon_wgs/6.0 + 30.5))
    idx_lat = int(round(lat_wgs/8.0 + 9.5))
    utm_lon = ((idx_lon-1) % 60) + 1
    utm_lat = fletter[idx_lat]
    return f'{utm_lon}{utm_lat}'

# returns bottom left corner (cell is 6x8 degrees)
def utm2wgs(lon_utm, lat_utm):
    idx_lon = lon_utm - 1
    idx_lat = rletter[lat_utm]
    lon_wgs = 6*(idx_lon - 30)
    lat_wgs = 8*(idx_lat - 10)
    return lon_wgs, lat_wgs

In [ ]:
utm_zones = [
                '45U',                        '50U','51U','52U','53U',
    '43T','44T','45T','46T','47T','48T','49T','50T','51T','52T','53T',
    '43S','44S','45S','46S','47S','48S','49S','50S','51S',
          '44R','45R','46R','47R','48R','49R','50R','51R',
                            '47Q','48Q','49Q','50Q'
]
utm_codes = [(int(z[:-1]), z[-1]) for z in utm_zones]
utm_base = [utm2wgs(lon, lat) for lon, lat in utm_codes]
utm_boxes = gpd.GeoSeries([make_rectangle(lon, lon+6, lat, lat+8) for lon, lat in utm_base])
utm_proj = {z: Proj(f'+proj=utm +zone={z}, +ellps=WGS84 +datum=WGS84 +units=m +no_defs') for z in utm_zones}
utm_func = lambda z, lon, lat, inverse=False: utm_proj[z](lon, lat, inverse=inverse)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4.5))
crop.plot(ax=ax);
utm_boxes.plot(ax=ax, color='g', alpha=0.2);
plt.axis('equal');

In [ ]:
rad_x, rad_y = 500000, 500000
utm_centers_wgs = [(lon+3, lat+4) for lon, lat in utm_base]
utm_centers_utm = [utm_func(z, lon, lat) for z, (lon, lat) in zip(utm_zones, utm_centers_wgs)]
utm_radboxs_utm = [(lon-rad_x, lon+rad_x, lat-rad_y, lat+rad_y) for lon, lat in utm_centers_utm]
utm_radrect_utm = [make_rectangle(*x, poly=False) for x in utm_radboxs_utm]
utm_radrect_wgs = [[utm_func(z, *y, inverse=True) for y in x] for z, x in zip(utm_zones, utm_radrect_utm)]
utm_radpoly_wgs = gpd.GeoSeries([Polygon(x) for x in utm_radrect_wgs])
fig, ax = plt.subplots(figsize=(6, 6))
crop.plot(ax=ax);
utm_radpoly_wgs.plot(ax=ax, color='g', alpha=0.2);
#utm_boxes.loc[:3].plot(ax=ax, color='r', alpha=0.2);
plt.axis('equal');

## Firm Acquisition

In [ ]:
# these should be WGS 84 datum (EPSG 3857)
x_lon, x_lat, proj = 116.390797, 39.916406, 'wgs-84' # beijing (forbidden city)
# x_lon, x_lat, proj = 116.406562, 39.882110, 'wgs' # beijing (temple of heaven)
# x_lon, x_lat, proj = 114.0665, 22.654437, 'bd-09' # huawei headquarters
# x_lon, x_lat, proj = 121.486958, 31.244277, 'wgs' # shanghai bund point - WGS84

In [ ]:
index = pd.read_csv('targets/google_scenes_2002_mincloud.csv')
x_pid = location_tools.find_scene(x_lon, x_lat, index)
im = Image.open(f'scenes/{x_pid}_B8.TIF')
im.resize((256, 256))

In [ ]:
def regularize(image, wins_lo=0.01, wins_hi=0.99, chop=True):
    arr = np.asarray(image).flatten()
    vlo, vhi = np.quantile(arr, [wins_lo, wins_hi])
    mat = np.clip(arr, vlo, vhi).reshape(image.size)
    return Image.fromarray(mat)

In [ ]:
size = 256
fig, ax = plt.subplots(figsize=(6, 6))
tile = location_tools.extract_satelite_tile(x_lon, x_lat, index, rad=256, size=size, proj=proj, resample=Image.LANCZOS)
ax.matshow(tile, cmap=cm.jet, origin='upper', interpolation='lanczos')
ax.scatter([size/2], [size/2], c='r');

## Firm Cluster

In [ ]:
census = pd.read_csv('index/census2004_mincloud2002.csv').dropna()

In [ ]:
utm_info = pd.read_csv('meta/utm_centers.csv', index_col='utm')

In [ ]:
pixel = 15
cmap = cm.jet

In [ ]:
extent = lambda s: [-pixel*s/2, pixel*s/2, -pixel*s/2, pixel*s/2]
decloud = lambda x, cut: np.choose(x<cut, [np.zeros_like(x), x])
amp = lambda x, alpha: 255*((x-np.min(x))/(np.max(x)-np.min(x)))**alpha

In [ ]:
# fidx = 271272 # huawei
fidx = 1082781 # Shanghai Happy Vacuum Cleaner Factory
rad, size = 1024, 256
ext = extent(rad)
finfo = census[census['id']==fidx].iloc[0]
sat = location_tools.extract_satelite_tile(finfo['lon_bd09'], finfo['lat_bd09'], rad, pid=finfo['prod_id'], size=size, proj='bd-09')
den = location_tools.extract_density_tile(finfo['lon_bd09'], finfo['lat_bd09'], rad=rad, size=size, proj='bd-09')
arr_sat, arr_den = np.asarray(sat), np.asarray(den)
arr_sat, arr_den = 1.4*amp(decloud(arr_sat, 100), 3.5), amp(arr_den, 0.7)
arr_mix = np.stack([arr_sat/255+2*arr_den/255, arr_sat/255, arr_sat/255], axis=2)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(arr_mix, interpolation='lanczos', extent=ext);

In [ ]:
fidx = 1082781 # Shanghai Happy Vacuum Cleaner Factory
# fidx = 271272 # Huawei

# fetch tile
rad, size = 1024, 256
ext = extent(rad)
finfo = census[census['id']==fidx].iloc[0]
den = location_tools.extract_density_tile(finfo['lon_bd09'], finfo['lat_bd09'], rad=rad, size=size, proj='bd-09', image=False)

# blur image at kmeter stdev
kmeter = 25
kpixel = kmeter/pixel
blur = gaussian_filter(den, sigma=kpixel)

# rectify image
alpha = 1/np.quantile(blur[blur>0], 0.95)
bend = 1 - 1/(1+alpha*blur)

# quantize image
im = Image.fromarray((255*bend).astype(np.uint8))
im = im.transpose(Image.FLIP_TOP_BOTTOM)
im = im.resize((size, size), resample=Image.LANCZOS)

# plot image
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(im, interpolation='lanczos', extent=ext);

In [ ]:
fidx = 1082781 # Shanghai Happy Vacuum Cleaner Factory
# fidx = 271272 # Huawei

# fetch tile
rad, size = 1024, 256
finfo = census[census['id']==fidx].iloc[0]
den = location_tools.extract_density_tile(finfo['lon_bd09'], finfo['lat_bd09'], rad=rad, size=size, proj='bd-09')
den